Un semplice pie chart sfruttando una query SQL con Couchbase Analytics e plotly. Questo notebook presuppone l'esistenza della collection `beer-sample`._default._default su analytics

In [35]:
user = "Administrator"
password = "password"
host = "localhost"

from couchbase.options import ClusterOptions
from couchbase.cluster import Cluster
from couchbase.auth import PasswordAuthenticator

cluster = Cluster("couchbase://" + host, ClusterOptions(
    PasswordAuthenticator(user, password)))

In [36]:

import plotly.graph_objects as go

def printPie(cluster, query, labels, values):
    result = cluster.query(query).execute()
    fig = go.Figure(data=[go.Pie(labels=[row[labels] for row in result if labels in row.keys()],
                                 values=[row[values] for row in result if values in row.keys()])])
    fig.show()

query = "SELECT COUNT(*) as count, country FROM `beer-sample` GROUP BY country ORDER BY count DESC LIMIT 10"

printPie(cluster, query, "country", "count")

Una applicazione Dash che permette di specificare il tipo di chart e la query da effettuare, e permette di fare query esplorative sui dati

In [37]:
#!pip install dash plotly jupyter_dash 
#!python -m pip install couchbase==3.2

import dash
from dash import dcc
from dash import html
from jupyter_dash import JupyterDash
import random
import json


cluster = Cluster("couchbase://" + host, ClusterOptions(
    PasswordAuthenticator(user, password)))

# Crea l'applicazione Dash
app = JupyterDash(__name__)

# Definisci il layout dell'applicazione
app.layout = html.Div(children=[
    html.H1(children='Couchbase Dash'),
    dcc.Graph(
        id='example-graph',
    ),
    html.Div(children=[
       html.Label('Insert a query to populate the chart. Label as `lab` what you want as label and as `val` what you want as values. Consider checking the validity of the attributes to avoid errors.'),
       dcc.Textarea(id='query-input', value='SELECT COUNT(*) as val, country as lab FROM `beer-sample`._default._default where ISSTRING(country) GROUP BY country ORDER BY val DESC LIMIT 10', style={'width': '100%', 'height': 100}),
                html.Label('Change the type of the chart:'),
        dcc.Dropdown(
            id='chart-type-dropdown',
            options=[
                {'label': 'Pie chart', 'value': 'pie'},
                {'label': 'Bar chart', 'value': 'bar'},
                {'label': 'Line chart', 'value': 'line'},
            ],
            value='pie',
            style={'width': '40%'}
        ),
        
        html.Button('Update chart', id='update-button', n_clicks=0),
        html.Br(),
        html.Br(),
        html.Label('Query your data:'),
        dcc.Textarea(id='query-exp', value='SELECT * FROM `beer-sample`._default._default LIMIT 1', style={'width': '100%', 'height': 100}),
        html.Button('Execute query', id='button'),
        dcc.Textarea(id='output', value='', style={'width': '100%', 'height': 500}),
    ])
])

# Definisci le callback per il pulsante di aggiornamento dei dati
@app.callback(
    dash.dependencies.Output('example-graph', 'figure'),
    [dash.dependencies.Input('update-button', 'n_clicks')],
    [dash.dependencies.State('query-input', 'value'),
    dash.dependencies.State('chart-type-dropdown', 'value')]
)
def update_data(n_clicks, query, chart_type):
    
    result = list(cluster.analytics_query(query))
    labels = [str(row["lab"]) for row in result]
    values = [row["val"] for row in result]
    if chart_type == 'pie':
         new_fig = go.Figure(data=[go.Pie(labels=labels,
                                 values=values)])
    elif chart_type == 'bar':
         new_fig = go.Figure(data=[go.Bar(x=labels,
                                 y=values)])
    elif chart_type == 'line':
        new_fig =  go.Figure(data=[go.Scatter(x=labels,
                                 y=values)])
    return new_fig

# Callback del pulsante
@app.callback(
    dash.dependencies.Output('output', 'value'),
    [dash.dependencies.Input('button', 'n_clicks')],
     [dash.dependencies.State('query-exp', 'value')]
)

  
# Indent keyword while dumping the
# data decides to what level
# spaces the user wants.

def show_document_structure(n_clicks, query):
    if n_clicks is None:
        return ''
    else:
        documents = list(cluster.analytics_query(query))
        return json.dumps(documents, indent = 1)
    
# Avvia l'applicazione
if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/



OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

Confrontiamo il response time tra una query effettuata su analytics e la stessa query sul servizio di query (che usa primary index)

In [38]:
%%time
result = list(cluster.analytics_query("SELECT COUNT(*) as val, country as lab FROM `beer-sample`._default._default where ISSTRING(country) GROUP BY country ORDER BY val DESC"))
labels = [str(row["lab"]) for row in result]
values = [row["val"] for row in result]

new_fig = go.Figure(data=[go.Bar(x=labels,
                                 y=values)])
new_fig.show()


CPU times: total: 0 ns
Wall time: 51.7 ms


In [42]:
%%time
result = list(cluster.query("SELECT COUNT(*) as val, country as lab FROM `beer-sample`._default._default where ISSTRING(country) GROUP BY country ORDER BY val DESC"))
labels = [str(row["lab"]) for row in result]
values = [row["val"] for row in result]

new_fig = go.Figure(data=[go.Bar(x=labels,
                                 y=values)])
new_fig.show()

CPU times: total: 0 ns
Wall time: 10.9 ms
